<a href="https://colab.research.google.com/github/alessioborgi/StyleAligned_DiffModels/blob/main/StyleAligned_Metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### SINGLE-STYLE METRICS

In [ ]:
# Download the file
!gdown --folder https://drive.google.com/drive/folders/1Rdb3XkwW1H_IMFVh3tShj4adh-XUNt03?usp=sharing

zsh:1: no matches found: https://drive.google.com/drive/folders/1Rdb3XkwW1H_IMFVh3tShj4adh-XUNt03?usp=sharing


In [ ]:
# Unzip the file if it's a zip
!unzip -q AlignZipFolder/OnlyPromptFile.zip

unzip:  cannot find or open AlignZipFolder/OnlyPromptFile.zip, AlignZipFolder/OnlyPromptFile.zip.zip or AlignZipFolder/OnlyPromptFile.zip.ZIP.


In [ ]:
# Install required libraries
!pip install transformers ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

  Using cached wcwidth-0.2.13-py2.py3-none-any.whl.metadata (14 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.0/43.0 kB 983.3 kB/s eta 0:00:000:00:01
Using cached wcwidth-0.2.13-py2.py3-none-any.whl (34 kB)
  Attempting uninstall: wcwidth
    Found existing installation: wcwidth 0.2.5
    Uninstalling wcwidth-0.2.5:
      Successfully uninstalled wcwidth-0.2.5

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
  Cloning https://github.com/openai/CLIP.git to /private/var/folders/mx/xlmhp5ln7l5g33gb6w32s46m0000gn/T/pip-req-build-e12_6icu
  Running command git clone --quiet https://github.com/openai/CLIP.git /private/var/folders/mx/xlmhp5ln7l5g33gb6w32s46m0000gn/T/pip-req-build-e12_6icu
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369520 sha256=04371c464313409990a7b4e

In [ ]:
import os
import torch
import clip
from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from transformers import AutoImageProcessor, AutoModel
from torchvision import transforms

# Load CLIP model and preprocess
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model, clip_preprocess = clip.load("ViT-B/32", device=device)

# Load DINO model and processor from Hugging Face
dino_processor = AutoImageProcessor.from_pretrained("facebook/dino-vitb8")
dino_model = AutoModel.from_pretrained("facebook/dino-vitb8").to(device)

def get_clip_similarity(image_path, prompt):
    image = clip_preprocess(Image.open(image_path)).unsqueeze(0).to(device)
    text = clip.tokenize([prompt]).to(device)

    with torch.no_grad():
        image_features = clip_model.encode_image(image)
        text_features = clip_model.encode_text(text)

    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    similarity = (image_features @ text_features.T).item()
    return similarity

def get_dino_embeddings(image_paths):
    images = [Image.open(image_path) for image_path in image_paths]
    inputs = dino_processor(images=images, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = dino_model(**inputs)
        embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()  # Taking the CLS token

    return embeddings

def compute_pairwise_similarity(embeddings):
    similarities = cosine_similarity(embeddings)
    upper_tri_indices = np.triu_indices_from(similarities, k=1)
    return similarities[upper_tri_indices].mean()

def process_folder(folder_path):
    subfolders = [f.path for f in os.scandir(folder_path) if f.is_dir()]
    clip_results = {'aligned': [], 'non_aligned': []}
    dino_results = {'aligned': [], 'non_aligned': []}

    for subfolder in subfolders:
        aligned_images = []
        non_aligned_images = []
        subfolder_name = os.path.basename(subfolder)

        for file in os.listdir(subfolder):
            if file.endswith(".jpg") or file.endswith(".png"):
                file_path = os.path.join(subfolder, file)
                if file.startswith("znon"):
                    non_aligned_images.append(file_path)
                else:
                    aligned_images.append(file_path)

        for image_path in aligned_images:
            image_name = os.path.basename(image_path)
            prompt = f"{image_name[:-4]}, {subfolder_name}"
            clip_similarity = get_clip_similarity(image_path, prompt)
            clip_results['aligned'].append(clip_similarity)

        for image_path in non_aligned_images:
            image_name = os.path.basename(image_path)[5:]
            prompt = f"{image_name[:-4]}, {subfolder_name}"
            clip_similarity = get_clip_similarity(image_path, prompt)
            clip_results['non_aligned'].append(clip_similarity)

        if aligned_images:
            aligned_embeddings = get_dino_embeddings(aligned_images)
            dino_similarity = compute_pairwise_similarity(aligned_embeddings)
            dino_results['aligned'].append(dino_similarity)

        if non_aligned_images:
            non_aligned_embeddings = get_dino_embeddings(non_aligned_images)
            dino_similarity = compute_pairwise_similarity(non_aligned_embeddings)
            dino_results['non_aligned'].append(dino_similarity)

    avg_clip_aligned = np.mean(clip_results['aligned'])
    avg_clip_non_aligned = np.mean(clip_results['non_aligned'])
    avg_dino_aligned = np.mean(dino_results['aligned'])
    avg_dino_non_aligned = np.mean(dino_results['non_aligned'])

    return {
        "clip_aligned": avg_clip_aligned,
        "clip_non_aligned": avg_clip_non_aligned,
        "dino_aligned": avg_dino_aligned,
        "dino_non_aligned": avg_dino_non_aligned
    }

# Example usage
folder_path = "OnlyPrompt"
results = process_folder(folder_path)
print(results)

100%|███████████████████████████████████████| 338M/338M [00:05<00:00, 61.8MiB/s]
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/453 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/343M [00:00<?, ?B/s]

Some weights of ViTModel were not initialized from the model checkpoint at facebook/dino-vitb8 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'clip_aligned': 0.34217529296875, 'clip_non_aligned': 0.3454345703125, 'dino_aligned': 0.5018175, 'dino_non_aligned': 0.34539917}


In [ ]:
results

{'clip_aligned': 0.34217529296875,
 'clip_non_aligned': 0.3454345703125,
 'dino_aligned': 0.5018175,
 'dino_non_aligned': 0.34539917}

### MULTI-STYLE DINO VIT-B/8

In [1]:
!git clone https://github.com/alessioborgi/StyleAlignedDiffModels.git

# Change directory to the cloned repository
%cd StyleAlignedDiffModels
%ls

Cloning into 'StyleAlignedDiffModels'...
remote: Enumerating objects: 1919, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 1919 (delta 21), reused 46 (delta 18), pack-reused 1868 (from 1)
Receiving objects: 100% (1919/1919), 1.26 GiB | 24.11 MiB/s, done.
Resolving deltas: 100% (791/791), done.
Updating files: 100% (454/454), done.
/content/StyleAlignedDiffModels
 audio/
 imgs/
 LICENSE
 music/
 README.md
 requirements.txt
 src/
 StyleAligned_ControlNet.ipynb
 StyleAligned_Explanation.ipynb
 StyleAligned_Focused_Attention_Explanation.ipynb
 StyleAligned_Metrics.ipynb
 StyleAligned_MultiReference.ipynb
 StyleAligned_MultiReference_Slerp.ipynb
 StyleAligned_MultiScale_AdaIN_Explanation.ipynb
 StyleAligned_Prompts.ipynb
'StyleAligned_Ref&Content(Audio).ipynb'
'StyleAligned_Ref&Content(Audio+Music).ipynb'
'StyleAligned_Ref&Content(Img+Audio).ipynb'
'StyleAligned_Ref&Content(Img+Audio+Music).ipynb'
'StyleAligned_Ref&Content(

In [2]:
# Download the file
!gdown --folder https://drive.google.com/drive/folders/1Rdb3XkwW1H_IMFVh3tShj4adh-XUNt03?usp=sharing

Retrieving folder contents
Processing file 1qQ7MkB89aJtzAeljDxcsDW86fvFsyWOx OnlyPromptFile.zip
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From (original): https://drive.google.com/uc?id=1qQ7MkB89aJtzAeljDxcsDW86fvFsyWOx
From (redirected): https://drive.google.com/uc?id=1qQ7MkB89aJtzAeljDxcsDW86fvFsyWOx&confirm=t&uuid=499d023e-2f1f-4bc9-8c49-5d872d8bc88c
To: /content/StyleAlignedDiffModels/AlignZipFolder/OnlyPromptFile.zip
100% 125M/125M [00:06<00:00, 19.8MB/s]
Download completed


In [3]:
# Unzip the file if it's a zip
!unzip -q AlignZipFolder/OnlyPromptFile.zip

In [4]:
# Install required libraries
!pip install transformers ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.0/43.0 kB 3.3 MB/s eta 0:00:00
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-82szrw17
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-82szrw17
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369490 sha256=be8de78deeb9b2a071c4ddf6837e82665af7c0fc24009407e04b5a6d2094f726
  Stored in directory: /tmp/pip-ephem-wheel-cache-6r0y20w0/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip


In [5]:
import numpy as np
import os
import torch
from PIL import Image
from transformers import AutoImageProcessor, AutoModel
from sklearn.metrics.pairwise import cosine_similarity

# Load DINO model and processor from Hugging Face
device = "cuda" if torch.cuda.is_available() else "cpu"
dino_processor = AutoImageProcessor.from_pretrained("facebook/dino-vitb8")
dino_model = AutoModel.from_pretrained("facebook/dino-vitb8").to(device)

def get_dino_embeddings(image_paths):
    """
    Extract DINO VIT-B/8 embeddings from a list of image paths.

    :param image_paths: List of paths to images.
    :return: List of embeddings corresponding to each image.
    """
    images = [Image.open(image_path) for image_path in image_paths]
    inputs = dino_processor(images=images, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = dino_model(**inputs)
        embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()  # Taking the CLS token

    return embeddings

def compute_similarity(generated_embedding, reference_embedding):
    """
    Compute the similarity between the generated image and a single reference style.

    :param generated_embedding: Embedding of the generated image (1D numpy array).
    :param reference_embedding: Embedding of the reference image (1D numpy array).
    :return: Similarity score.
    """
    similarity = cosine_similarity(generated_embedding.reshape(1, -1), reference_embedding.reshape(1, -1)).item()
    return similarity

def process_main_folder(main_folder, reference_image_paths):
    """
    Process the main folder and compute the weighted DINO metric for each subfolder.
    """
    final_results = {}

    subfolders = [f.path for f in os.scandir(main_folder) if f.is_dir()]

    for subfolder in subfolders:
        subfolder_name = os.path.basename(subfolder)
        # Extract weights from the subfolder name
        try:
            parts = subfolder_name.split('_')
            weight1 = float(parts[-2])
            weight2 = float(parts[-1])
            weights = [weight1, weight2]
        except Exception as e:
            print(f"Error parsing weights from {subfolder_name}: {e}")
            continue

        # Print the header with reference image information
        print(f"Results for {subfolder_name} (reference_image 1: {os.path.basename(reference_image_paths[0])}, reference_image 2: {os.path.basename(reference_image_paths[1])}):")

        # Calculate the mean DINO similarity for all images in the subfolder w.r.t. both references
        similarities_ref1 = []
        similarities_ref2 = []

        for image_file in os.listdir(subfolder):
            if image_file.endswith(".jpg") or image_file.endswith(".png"):
                # Skip the reference image file
                if image_file == "ref_img.png":
                    continue

                image_path = os.path.join(subfolder, image_file)
                generated_embedding = get_dino_embeddings([image_path])[0]
                reference_embeddings = get_dino_embeddings(reference_image_paths)

                similarity_to_ref1 = compute_similarity(generated_embedding, reference_embeddings[0])
                similarity_to_ref2 = compute_similarity(generated_embedding, reference_embeddings[1])

                similarities_ref1.append(similarity_to_ref1)
                similarities_ref2.append(similarity_to_ref2)

                # Print similarity results for each image
                print(f"Image: {image_file}, Similarity to Reference 1: {similarity_to_ref1}, Similarity to Reference 2: {similarity_to_ref2}")

        # Calculate the mean similarities for the subfolder
        mean_similarity_ref1 = np.mean(similarities_ref1)
        mean_similarity_ref2 = np.mean(similarities_ref2)

        # Compute the final weighted multi-style DINO metric
        weighted_multi_style_dino = mean_similarity_ref1 * weight1 + mean_similarity_ref2 * weight2

        final_results[subfolder_name] = {
            "mean_similarity_ref1": mean_similarity_ref1,
            "mean_similarity_ref2": mean_similarity_ref2,
            "weighted_multi_style_dino": weighted_multi_style_dino
        }

        print(f"Mean Similarity to Reference 1: {mean_similarity_ref1}")
        print(f"Mean Similarity to Reference 2: {mean_similarity_ref2}")
        print(f"Weighted Multi-Style DINO Metric: {weighted_multi_style_dino}\n")

    return final_results



preprocessor_config.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/453 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/343M [00:00<?, ?B/s]

Some weights of ViTModel were not initialized from the model checkpoint at facebook/dino-vitb8 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#### LINEAR WEIGHTED BLENDING

##### GUIDANCE 10

In [6]:
# Example usage
main_folder = "./imgs/Multi_Reference/Weighted_Latent_Space/medieval_picasso_Guidance_10"
reference_image_paths = ["./imgs/medieval-bed.jpeg", "./imgs/Picasso_Smoking_Water_Pipe.jpeg"]
results = process_main_folder(main_folder, reference_image_paths)

# Print or save the results
for subfolder, metrics in results.items():
    print(f"Final Results for {subfolder}:")
    print(f"Mean Similarity to Reference 1: {metrics['mean_similarity_ref1']}")
    print(f"Mean Similarity to Reference 2: {metrics['mean_similarity_ref2']}")
    print(f"Weighted Multi-Style DINO Metric: {metrics['weighted_multi_style_dino']}\n")

Results for medieval_picasso_1_0 (reference_image 1: medieval-bed.jpeg, reference_image 2: Picasso_Smoking_Water_Pipe.jpeg):
Image: scarecrow.png, Similarity to Reference 1: 0.24083636701107025, Similarity to Reference 2: 0.37675100564956665
Image: lighthouse.png, Similarity to Reference 1: 0.3318901062011719, Similarity to Reference 2: 0.33703693747520447
Image: mountain_landscape.png, Similarity to Reference 1: 0.34509122371673584, Similarity to Reference 2: 0.4812157452106476
Image: castle.png, Similarity to Reference 1: 0.2660902142524719, Similarity to Reference 2: 0.31534716486930847
Image: air_baloon.png, Similarity to Reference 1: 0.3823093771934509, Similarity to Reference 2: 0.2968261241912842
Image: golf.png, Similarity to Reference 1: 0.3861086070537567, Similarity to Reference 2: 0.4416363835334778
Mean Similarity to Reference 1: 0.3253876492381096
Mean Similarity to Reference 2: 0.37480222682158154
Weighted Multi-Style DINO Metric: 0.3253876492381096

Results for medieval

##### GUIDANCE 25

In [8]:
# Example usage
main_folder = "./imgs/Multi_Reference/Weighted_Latent_Space/medieval_picasso_Guidance_25"
reference_image_paths = ["./imgs/medieval-bed.jpeg", "./imgs/Picasso_Smoking_Water_Pipe.jpeg"]
results = process_main_folder(main_folder, reference_image_paths)

# Print or save the results
for subfolder, metrics in results.items():
    print(f"Final Results for {subfolder}:")
    print(f"Mean Similarity to Reference 1: {metrics['mean_similarity_ref1']}")
    print(f"Mean Similarity to Reference 2: {metrics['mean_similarity_ref2']}")
    print(f"Weighted Multi-Style DINO Metric: {metrics['weighted_multi_style_dino']}\n")

Results for medieval_picasso_1_0 (reference_image 1: medieval-bed.jpeg, reference_image 2: Picasso_Smoking_Water_Pipe.jpeg):
Image: mushrooms.png, Similarity to Reference 1: 0.3140500783920288, Similarity to Reference 2: 0.27019909024238586
Image: scarecrow.png, Similarity to Reference 1: 0.37879446148872375, Similarity to Reference 2: 0.37161117792129517
Image: lighthouse.png, Similarity to Reference 1: 0.42154496908187866, Similarity to Reference 2: 0.3311164975166321
Image: castle.png, Similarity to Reference 1: 0.27519989013671875, Similarity to Reference 2: 0.3981720209121704
Image: air_baloon.png, Similarity to Reference 1: 0.30036813020706177, Similarity to Reference 2: 0.29647260904312134
Image: mountains.png, Similarity to Reference 1: 0.23457500338554382, Similarity to Reference 2: 0.4646250903606415
Image: golf.png, Similarity to Reference 1: 0.3399296700954437, Similarity to Reference 2: 0.43160536885261536
Mean Similarity to Reference 1: 0.3234946003981999
Mean Similarity 

#### SLERP BLENDING

##### GUIDANCE 10

In [10]:
# Example usage
main_folder = "./imgs/Multi_Reference/Weighted_Latent_Space_SLERP/medieval_picasso_guidance_10"
reference_image_paths = ["./imgs/medieval-bed.jpeg", "./imgs/Picasso_Smoking_Water_Pipe.jpeg"]
results = process_main_folder(main_folder, reference_image_paths)

# Print or save the results
for subfolder, metrics in results.items():
    print(f"Final Results for {subfolder}:")
    print(f"Mean Similarity to Reference 1: {metrics['mean_similarity_ref1']}")
    print(f"Mean Similarity to Reference 2: {metrics['mean_similarity_ref2']}")
    print(f"Weighted Multi-Style DINO Metric: {metrics['weighted_multi_style_dino']}\n")

Results for medieval_picasso_1_0 (reference_image 1: medieval-bed.jpeg, reference_image 2: Picasso_Smoking_Water_Pipe.jpeg):
Image: scarecrow.png, Similarity to Reference 1: 0.3909943401813507, Similarity to Reference 2: 0.3329729437828064
Image: lighthouse.png, Similarity to Reference 1: 0.39169394969940186, Similarity to Reference 2: 0.3081279993057251
Image: castle.png, Similarity to Reference 1: 0.33056825399398804, Similarity to Reference 2: 0.3134803771972656
Image: air_baloon.png, Similarity to Reference 1: 0.3788291811943054, Similarity to Reference 2: 0.2098432332277298
Image: mountains.png, Similarity to Reference 1: 0.3864254355430603, Similarity to Reference 2: 0.27427464723587036
Image: golf.png, Similarity to Reference 1: 0.45199745893478394, Similarity to Reference 2: 0.4151020646095276
Mean Similarity to Reference 1: 0.38841810325781506
Mean Similarity to Reference 2: 0.30896687755982083
Weighted Multi-Style DINO Metric: 0.38841810325781506

Results for medieval_picasso

##### GUIDANCE 25

In [12]:
# Example usage
main_folder = "./imgs/Multi_Reference/Weighted_Latent_Space_SLERP/medieval_picasso_guidance_25"
reference_image_paths = ["./imgs/medieval-bed.jpeg", "./imgs/Picasso_Smoking_Water_Pipe.jpeg"]
results = process_main_folder(main_folder, reference_image_paths)

# Print or save the results
for subfolder, metrics in results.items():
    print(f"Final Results for {subfolder}:")
    print(f"Mean Similarity to Reference 1: {metrics['mean_similarity_ref1']}")
    print(f"Mean Similarity to Reference 2: {metrics['mean_similarity_ref2']}")
    print(f"Weighted Multi-Style DINO Metric: {metrics['weighted_multi_style_dino']}\n")

Results for medieval_picasso_1_0 (reference_image 1: medieval-bed.jpeg, reference_image 2: Picasso_Smoking_Water_Pipe.jpeg):
Image: scarecrow.png, Similarity to Reference 1: 0.44854736328125, Similarity to Reference 2: 0.39558112621307373
Image: lighthouse.png, Similarity to Reference 1: 0.42154496908187866, Similarity to Reference 2: 0.3311164975166321
Image: castle.png, Similarity to Reference 1: 0.44291308522224426, Similarity to Reference 2: 0.32869166135787964
Image: air_baloon.png, Similarity to Reference 1: 0.38328251242637634, Similarity to Reference 2: 0.24222637712955475
Image: mountains.png, Similarity to Reference 1: 0.3559674024581909, Similarity to Reference 2: 0.294542133808136
Image: golf.png, Similarity to Reference 1: 0.44195836782455444, Similarity to Reference 2: 0.43918943405151367
Mean Similarity to Reference 1: 0.41570228338241577
Mean Similarity to Reference 2: 0.338557871679465
Weighted Multi-Style DINO Metric: 0.41570228338241577

Results for medieval_picasso_